In [1]:
import sys
sys.path.append('../../30_data_tools/')
from pathlib import Path
from random import choices
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
import sqlite3
import re
import cv2
# import YOLO model
from ultralytics import YOLO

In [2]:
from helper import load_dotenv
from get_labelstudio_data import get_results_of_project

In [3]:
TILE_SIZE = 280
MIN_MIDTONE_SHARE = 0.05

In [4]:
dotenv = load_dotenv()

In [5]:
con = sqlite3.connect( dotenv['DB_PATH'] )

In [6]:
exclude_jobs = [
    '23-10-03_Testformen',
    '23-10-18_farbe',
    '23-10-17_blur',
    '23-10-19_farbe'
]

In [7]:
results = [
    r for r in get_results_of_project(1)
    if r['rectanglelabels'][0] not in ['potential_moire'] and True not in [r['img_name'].startswith(ej) for ej in exclude_jobs]
]

In [8]:
pages = pd.read_sql(
    '''
        SELECT * FROM related_file rf 
        WHERE variant_name = 'ps2400dpi150lpi' AND "type" = '4c_600'
    ''',
    con
)
pages.loc[
    :,
    'dpi'
] = pd.to_numeric(pages.filename.str.extract(r'.+\.4c_(\d+)\.jpg')[0])

In [9]:
model = YOLO(dotenv['MODEL_DIR'] / 'yolov8_moires_24-02-22.pt')

In [10]:
def step_1_classifier( img ):
    relevant_tiles = []    
    left = 0
    top = 0
    
    while left < img.size[0] or top < img.size[1]:
        tile = img.crop((
            left,top,
            left+TILE_SIZE,top+TILE_SIZE
        ))
        np_tile = np.array(tile)
        is_relevant = False
        
        for i in range(np_tile.shape[2]):
            sep = np_tile[:,:,i]
        
            midtone_share = sep[(sep > 8) & (sep < 247)].shape[0] / (sep.shape[0] * sep.shape[1])
            if midtone_share > MIN_MIDTONE_SHARE:
                is_relevant = True
                break
    
        if is_relevant:
            relevant_tiles.append(((left, top),tile))
    
        if left < img.size[0]:
            left += round(TILE_SIZE / 2)
        elif top < img.size[1]:
            left = 0
            top += round(TILE_SIZE / 2)
        else:
            break

    return relevant_tiles

In [11]:
def step_2_classifier( tiles ):
    classifications = []

    for pos,tile in tiles:
        tile_to_process = Image.fromarray(np.array(tile)[:,:,3]).convert('RGB')
        
        pred = model.predict(tile_to_process, verbose=False)[0].probs.data
        target_label = 1 - int(torch.argmax(pred))
        classifications.append((
            pos,
            tile,
            target_label,
            pred
        ))
    
    return classifications

In [12]:
def get_out_map( img, relevant_tiles ):
    out_img = np.zeros((img.size[1], img.size[0], 4))
    colors = [
        (0,0,255),
        (255,0,0)
    ]
    
    for pos,tile,label,pred in relevant_tiles:
        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            0
        ] += 1

        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            1
        ] += float(pred[1])

        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            2
        ] += float(pred[0])

    out_img /= 4

    level_1_img = Image.fromarray((out_img[:,:,0] * 255).astype('uint8'))
    level_2_img = Image.fromarray((out_img[:,:,1] * 255).astype('uint8'))
    level_3_img = Image.fromarray((out_img[:,:,2] * 255).astype('uint8'))
    level_4_img = out_img[:,:,2] * 255
    level_4_img[(out_img[:,:,2] < out_img[:,:,1]) | (out_img[:,:,2] < 0.5)] = 0
    level_4_img = Image.fromarray(level_4_img.astype('uint8'))
    
    return level_1_img, level_2_img, level_3_img, level_4_img

In [13]:
def create_result_map( img_size, tiles ):
    result_map = np.zeros((img_size[1],img_size[0])).astype('float32')
    
    for pos,tile,label,pred in tiles:
        if pred[1] < pred[0]:
            result_map[
                pos[1]:pos[1]+TILE_SIZE,
                pos[0]:pos[0]+TILE_SIZE,
            ] += float(pred[0]) / 4
            # / 4 weil max. vier tiles die Kachel bestimmen

    return result_map

In [14]:
def get_moires_of_page( row, results ):
    relevant_moires = []
    
    for r in results:
        if re.match(f'^{ row.job }\..+?\.{ row.pdf_filename }\..+', r['img_name']):
            dpi = int( re.match(r'.+\.4c_(\d+)\.jpg', r['img_name']).groups()[0] )
            out_box = [r['value']['x'],r['value']['y'],r['value']['width'],r['value']['height']]
            
            # box umrechnen
            if target_dpi != dpi:
                out_box = [round(val * (target_dpi / dpi)) for val in out_box]
    
            relevant_moires.append(out_box)

    return relevant_moires

In [15]:
def get_intersection_over_union( box_a, box_b ):
    intersection_box = [
        max(box_a[0],box_b[0]),
        max(box_a[1],box_b[1]),
        min(box_a[0]+box_a[2],box_b[0]+box_b[2]),
        min(box_a[1]+box_a[3],box_b[1]+box_b[3]),
    ]

    if intersection_box[2] - intersection_box[0] < 0 or intersection_box[3] - intersection_box[1] < 0:
        return 0

    intersection = (intersection_box[2] - intersection_box[0]) * (intersection_box[3] - intersection_box[1])
    
    union_box = [
        min(box_a[0],box_b[0]),
        min(box_a[1],box_b[1]),
        max(box_a[0]+box_a[2],box_b[0]+box_b[2]),
        max(box_a[1]+box_a[3],box_b[1]+box_b[3]),
    ]
    union = (union_box[2] - union_box[0]) * (union_box[3] - union_box[1])

    return intersection / union

In [16]:
def get_result_boxes( result_map, threshold=0.5 ):
    thresh = np.zeros(result_map.shape).astype('uint8')
    thresh[result_map > threshold] = 255

    (numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(
    	thresh, 4, cv2.CV_32S
    )

    return [
        [b[0],b[1],b[2],b[3]]
        for b in stats[1:]
    ]

In [17]:
def draw_bounding_boxes( img, moire_boxes, predicted_boxes ):
    colors = {
        "target" : "green",
        "predicted" : "red"
    }    

    img_out = img.copy().convert('RGB')
    draw = ImageDraw.Draw(img_out) 

    for b in moire_boxes:
        draw.rectangle([b[0],b[1],b[0]+b[2],b[1]+b[3]], outline=colors['target'], width=10) 

    for b in predicted_boxes:
        draw.rectangle([b[0],b[1],b[0]+b[2],b[1]+b[3]], outline=colors['predicted'], width=10) 

    
    return img_out

In [18]:
target_dpi = dotenv['TRAIN_DATA_DPI']

In [19]:
relevant_idx = []

for r in results:
    res = re.match(f'^(.+?)\..+?\.(.+?)\.4c_\d+\.jpg', r['img_name'])
    try:
        job, pdf_filename = res.groups()
    
        relevant_idx.append(
            pages.loc[
                (pages.job == job) &
                (pages.pdf_filename == pdf_filename)
            ].iloc[0].name
        )
    except:
        pass

relevant_idx = list(set(relevant_idx))

In [ ]:
0 / 0

In [21]:
out = []

In [82]:
samples = pd.concat([
    pages.loc[
        pages.index.isin([o[0] for o in out]) == False
    ].sample(n=40),
    pages.loc[
        (pages.index.isin([o[0] for o in out]) == False) &
        (pages.index.isin(choices(relevant_idx, k=20)))
    ]
]).sample(frac=1)

In [83]:
for i in tqdm(range(samples.shape[0])):
    sample = samples.iloc[i]
    
    img_path = dotenv['DATA_DIR'] / sample.job / sample.variant_name / sample.filename
    img = Image.open(img_path)
    img = img.resize((
        round(img.size[0] * (target_dpi / sample.dpi)),
        round(img.size[1] * (target_dpi / sample.dpi))
    ))
    relevant_tiles = step_1_classifier( img )
    classifications = step_2_classifier( relevant_tiles )

    result_map = create_result_map( img.size, classifications )
    relevant_moires = get_moires_of_page( sample, results )

    predicted_label = int(result_map[result_map > 0.5].shape[0] > 0)
    label = int(len(relevant_moires) > 0)

    if label == 1 or predicted_label == 1:
        result_boxes = get_result_boxes( result_map ) if predicted_label == 1 else []

        if len(result_boxes) > 0:
            out.append((
                sample.name, predicted_label, label, [max([get_intersection_over_union(r,p) for p in result_boxes]) for r in relevant_moires]
            ))
        else:
            out.append((
                sample.name, predicted_label, label, []
            ))
    
        draw_bounding_boxes( img, relevant_moires, result_boxes ).save(f'./temp/{ sample.filename }')
    else:
        out.append((
            sample.name, predicted_label, label, []
        ))

  0%|          | 0/41 [00:00<?, ?it/s]

In [84]:
pages.loc[
    [o[0] for o in out],
    'predicted_label'
] = [o[1] for o in out]

pages.loc[
    [o[0] for o in out],
    'label'
] = [o[2] for o in out]

In [85]:
pages.loc[
    pd.isna(pages.predicted_label) == False
].to_pickle('./temp/out.pkl')

In [86]:
TP = pages.loc[
    (pages.label == 1) &
    (pages.predicted_label == 1)
].shape[0]
TN = pages.loc[
    (pages.label == 0) &
    (pages.predicted_label == 0)
].shape[0]
FP = pages.loc[
    (pages.label == 0) &
    (pages.predicted_label == 1)
].shape[0]
FN = pages.loc[
    (pages.label == 1) &
    (pages.predicted_label == 0)
].shape[0]

In [87]:
recall = TP / (TP + FN)
precision = TP / (TP + FP)
correct_values = TP + TN
accuracy = correct_values / len(out)

In [88]:
TP, TN, FP, FN

(35, 140, 57, 0)

In [89]:
accuracy, recall, precision

(0.7446808510638298, 1.0, 0.3804347826086957)

In [ ]:
0 / 0

In [92]:
samples = pages.loc[
    (pages.predicted_label == 1) & (pages.label == 0)
]

In [93]:
for i in tqdm(range(samples.shape[0])):
    sample = samples.iloc[i]
    resolution = sample.dpi
    img_path = dotenv['DATA_DIR'] / sample.job / sample.variant_name / sample.filename

    img = Image.open(img_path)
    img = img.resize((
        round(img.size[0] * (target_dpi / resolution)),
        round(img.size[1] * (target_dpi / resolution))
    ))
    relevant_tiles = step_1_classifier( img )
    classifications = step_2_classifier( relevant_tiles )

    # blend img erzeugen
    TARGET_OUT_HEIGHT = 1000
    colors=['red','green','blue','orange']
    l_images = get_out_map( img, classifications )
    blended = Image.new(mode="RGB", size=(img.size[0] * len(l_images), img.size[1]))

    for i in range(len(l_images)):
        l_img = ImageEnhance.Brightness(l_images[i]).enhance(0.5)
        overlay = Image.new('RGB', l_img.size, color=colors[i])
        l_rgb = img.convert('RGB')
        l_rgb.paste(
            overlay,
            (0,0),
            mask=l_img
        )

        blended.paste(
            l_rgb,
            (img.size[0] * i,0)
        )

    blended = blended.resize((
        round(TARGET_OUT_HEIGHT / blended.size[1] * blended.size[0]),
        TARGET_OUT_HEIGHT
    ))
    
    blended.save( Path('./result_images') / f"separated_{sample.name}.jpg", progressive=True )

  0%|          | 0/57 [00:00<?, ?it/s]